In [1]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


In [2]:
import torch
import pandas as pd
import numpy as np

In [3]:
!pip install transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 113.9 MB/s eta 0:00:00


In [4]:
df = pd.read_csv('/content/Complaint data annotation (explain)_updated - cd (1).csv')

In [5]:
df.keys()

Index(['id', 'tweet', 'label', 'domain', 'sentiment', 'emotion', 'Severity',
       'Explain'],
      dtype='object')

In [6]:
df.head()

,id,tweet,label,domain,sentiment,emotion,Severity,Explain
0,1,@FC_HELP can I return online purchases to a Ho...,0,apparel,Neutral,other,0,can I return online purchases to a House of Fr...
1,2,@FC_Help Hi - I'm writing a piece for MSN Him ...,0,apparel,Positive,other,0,Hi - I'm writing a piece for MSN Him and wonde...
2,3,@FC_Help i need to check my order,0,apparel,Neutral,other,0,i need to check my order
3,4,@FC_Help I need to get in contact with someone...,1,apparel,Neutral,other,1,I need to get in contact with someone regardin...
4,5,@FC_Help How can I get a hold of you so we can...,0,apparel,Negative,other,0,How can I get a hold of you so we can discuss ...


In [7]:
domain_dict = {
    "other" : 0,
    "services" : 1,
    "random_reply" : 2,
    "software" : 3,
    "retail" : 4,
    "random_tweet" : 5,
    "transport" : 6,
    "cars" : 7,
    "food" : 8,
    "apparel" : 9,
    "electronics" : 10
}

# note to take string lower
senti_dict = {
    'negative' : 0,
    'positive' : 1,
    'neutral' : 2
}

emo_dict = {
    'sadness' : 0, 
    'joy' : 1, 
    'other' : 2, 
    'anger' : 3, 
    'disgust' : 4, 
    'surprise' : 5, 
    'fear' : 6
}

In [8]:
label = []
domain = []
emotion = []
sentiment = []
severity = []
explain = []

for i in range(len(df)):
  if (pd.isna(df['label'][i]) or pd.isna(df['domain'][i]) or pd.isna(df['emotion'][i]) or pd.isna(df['sentiment'][i]) or pd.isna(df['Severity'][i]) or pd.isna(df['Explain'][i])):
    continue
  label.append(df['label'][i])
  domain.append(domain_dict[df['domain'][i]])
  emotion.append(emo_dict[df['emotion'][i]])
  sentiment.append(senti_dict[(df['sentiment'][i]).lower()])
  severity.append(df['Severity'][i])
  explain.append(df['Explain'][i])

In [9]:
from sklearn.model_selection import train_test_split

label_train, label_test, domain_train, domain_test, emotion_train, emotion_test, sentiment_train, sentiment_test, severity_train, severity_test, explain_train, explain_test = train_test_split(label, domain, emotion, sentiment, severity, explain, test_size = 0.2, random_state = 42, shuffle = True)
label_train, label_val, domain_train, domain_val, emotion_train, emotion_val, sentiment_train, sentiment_val, severity_train, severity_val, explain_train, explain_val = train_test_split(label_train, domain_train, emotion_train, sentiment_train, severity_train, explain_train, test_size = 0.2, random_state = 42, shuffle = True)

In [10]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def preprocess_data(text):
  g_input_ids = []
  g_attention_mask = []

  for line in text:
    inputs = tokenizer(line, return_tensors="pt")

    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    size = input_ids.shape[1]

    if size < 60:
      input_ids = torch.cat((input_ids, torch.zeros(1, 60-size)), dim = 1)
      attention_mask = torch.cat((attention_mask, torch.zeros(1, 60-size)), dim = 1)
    else:
      input_ids = input_ids[:, :60]
      attention_mask = attention_mask[:, :60]
    
    # Why we use np.array()
    g_input_ids.append(np.array(input_ids.reshape(-1)))
    g_attention_mask.append(np.array(attention_mask.reshape(-1)))

  g_input_ids = torch.tensor(g_input_ids)
  g_attention_mask = torch.tensor(g_attention_mask)

  return g_input_ids, g_attention_mask

In [11]:
input_ids_train, attention_mask_train = preprocess_data(explain_train)
input_ids_val, attention_mask_val = preprocess_data(explain_val)
input_ids_test, attention_mask_test = preprocess_data(explain_test)

<ipython-input-10-eb1c066b164e>:28: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  g_input_ids = torch.tensor(g_input_ids)


In [12]:
class Complaint_dataset():
  def __init__(self, input_ids, attention_mask, label, domain, emotion, sentiment, severity):
    self.input_ids = input_ids
    self.attention_mask = attention_mask
    self.label = label
    self.domain = domain
    self.emotion = emotion
    self.sentiment = sentiment
    self.severity = severity
  def __len__(self):
    return len(self.input_ids)
  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()
    
    sample = {
        "input_ids" : torch.tensor(self.input_ids[idx]).long().to(device),
        "attention_mask" : torch.tensor(self.attention_mask[idx]).long().to(device),
        "label" : torch.tensor(self.label[idx]).long().to(device),
        "domain" : torch.tensor(self.domain[idx]).long().to(device),
        "emotion" : torch.tensor(self.emotion[idx]).long().to(device),
        "sentiment" : torch.tensor(self.sentiment[idx]).long().to(device),
        "severity" : torch.tensor(self.severity[idx]).long().to(device)
    }

    return sample

In [13]:
from torch.utils.data import DataLoader

complaint_train = Complaint_dataset(input_ids_train, attention_mask_train, label_train, domain_train, emotion_train, sentiment_train, severity_train)
train_dataloader = DataLoader(complaint_train, batch_size = 32, shuffle = False)

complaint_val = Complaint_dataset(input_ids_val, attention_mask_val, label_val, domain_val, emotion_val, sentiment_val, severity_val)
val_dataloader = DataLoader(complaint_val, batch_size = 32, shuffle = False)

complaint_test = Complaint_dataset(input_ids_test, attention_mask_test, label_test, domain_test, emotion_test, sentiment_test, severity_test)
test_dataloader = DataLoader(complaint_test, batch_size = 32, shuffle = False)

In [14]:
import torch.nn as nn
from transformers import BertModel

class Sentiment(nn.Module):
  def __init__(self, classes):
    super(Sentiment, self).__init__()
    self.senti_a = nn.Linear(768, 512).to(device)
    self.senti_b = nn.Linear(512, 256).to(device)
    self.senti_c = nn.Linear(256, classes).to(device)
    self.relu = nn.ReLU()
    self.softmax = nn.Softmax(dim = 1).to(device)
  def forward(self, bert_embed):
    senti_a = self.relu(self.senti_a(bert_embed))
    senti_b = self.relu(self.senti_b(senti_a))
    senti_out = self.softmax(self.senti_c(senti_b))
    return senti_a, senti_b, senti_out

class Emotion(nn.Module):
  def __init__(self, classes):
    super(Emotion, self).__init__()
    self.emo_a = nn.Linear(768, 512).to(device)
    self.emo_b = nn.Linear(512, 256).to(device)
    self.emo_c = nn.Linear(256, classes).to(device)
    self.relu = nn.ReLU()
    self.softmax = nn.Softmax(dim = 1).to(device)
  def forward(self, bert_embed):
    emo_a = self.relu(self.emo_a(bert_embed))
    emo_b = self.relu(self.emo_b(emo_a))
    emo_out = self.softmax(self.emo_c(emo_b))
    return emo_a, emo_b, emo_out

class Complaint(nn.Module):
  def __init__(self):
    super(Complaint, self).__init__()
    self.bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)
    self.senti = Sentiment(classes = 3)
    self.emo = Emotion(classes = 7)

    self.senti.load_state_dict(torch.load("bert_sent.pt"))
    self.emo.load_state_dict(torch.load("bert_emo.pt"))

    self.central_b = nn.Linear(512, 256).to(device)
    self.central_c = nn.Linear(256, 128).to(device)
    self.central_comp = nn.Linear(128, 2).to(device)
    self.central_sev = nn.Linear(128, 5).to(device)
    self.relu = nn.ReLU()
    self.softmax = nn.Softmax(dim = 1).to(device)
  def forward(self, input_ids, attention_mask):
    bert_embed = self.bert_model(input_ids = input_ids.to(device), attention_mask = attention_mask.to(device)).pooler_output.to(device)

    for p in self.senti.parameters():
      p.require_grads = False

    for p in self.emo.parameters():
      p.require_grads = False

    senti_a, senti_b, senti_out = self.senti(bert_embed)
    emo_a, emo_b, emo_out = self.emo(bert_embed)

    batch_size = emo_a.shape[0]

    #parameters
    self.a = nn.ParameterList([nn.Parameter(torch.rand(1).to(device)) for i in range(4)])
    self.b = nn.ParameterList([nn.Parameter(torch.rand(1).to(device)) for i in range(4)])
    self.c = nn.ParameterList([nn.Parameter(torch.rand(1).to(device)) for i in range(4)])

    central_a = torch.zeros(batch_size, 512).to(device)
    wsum_a = self.a[0].expand_as(central_a)*central_a + self.b[0].expand_as(emo_a)*emo_a + self.c[0].expand_as(senti_a)*senti_a

    central_b = self.relu(self.central_b(wsum_a)).to(device)
    wsum_b = self.a[1].expand_as(central_b)*central_b + self.b[1].expand_as(emo_b)*emo_b + self.c[1].expand_as(senti_b)*senti_b

    central_c = self.relu(self.central_c(wsum_b)).to(device)

    # now the emo and senti are done with the output so they have their classes size
    # we need to concat them
    emo_out = torch.cat((emo_out, torch.zeros(batch_size, 128-emo_out.shape[1]).to(device)), dim = 1).to(device)
    senti_out = torch.cat((senti_out, torch.zeros(batch_size, 128-senti_out.shape[1]).to(device)), dim = 1).to(device)
    # since we have to concatenate horizontally on all the 32 samples in the batch

    wsum_out = self.a[2].expand_as(central_c)*central_c + self.b[2].expand_as(emo_out)*emo_out + self.c[2].expand_as(senti_out)*senti_out

    central_comp = self.softmax(self.central_comp(wsum_out)).to(device)
    sev_out = self.softmax(self.central_sev(wsum_out)).to(device)

    return bert_embed, central_comp, emo_out, senti_out, sev_out

    # need to add alphas and then do the data processing

In [15]:
import torch.nn.functional as F

class SupConLoss(nn.Module):

    def __init__(self):
        super().__init__()
        self.xent_loss = nn.CrossEntropyLoss()
        self.alpha = 0.5
        self.temp = 0.1

    def nt_xent_loss(self, anchor, target, labels):
        with torch.no_grad():
            labels = labels.unsqueeze(-1)
            mask = torch.eq(labels, labels.transpose(0, 1))
            mask = mask ^ torch.diag_embed(torch.diag(mask))

        anchor_dot_target = torch.einsum('bd,cd->bc', anchor, target) / self.temp

        anchor_dot_target = anchor_dot_target - torch.diag_embed(torch.diag(anchor_dot_target))

        logits_max, _ = torch.max(anchor_dot_target, dim=1, keepdim=True)
        logits = anchor_dot_target - logits_max.detach()

        exp_logits = torch.exp(logits)

        logits = logits * mask
        log_prob = logits - torch.log(exp_logits.sum(dim=1, keepdim=True) + 1e-12)

        mask_sum = mask.sum(dim=1)
        mask_sum = torch.where(mask_sum == 0, torch.ones_like(mask_sum), mask_sum)

        pos_logits = (mask * log_prob).sum(dim=1) / mask_sum.detach()
        loss = -1 * pos_logits.mean()
        return loss

    def forward(self, output, predicted, targets):
        normed_output = F.normalize(output, dim=-1)
        ce_loss = (1 - self.alpha) * self.xent_loss(predicted, targets)
        cl_loss = self.alpha * self.nt_xent_loss(normed_output, normed_output, targets)
        return cl_loss + ce_loss

In [16]:
BERT_MODEL = BertModel.from_pretrained('bert-base-uncased').to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
sentiment_model = Sentiment(classes = 3)
sentiment_model = sentiment_model.to(device)
sentiment_model.train()

loss_func = SupConLoss()
optimizer = torch.optim.Adam(sentiment_model.parameters(), lr = 1e-5, weight_decay = 0)
epochs = 20
min_val_loss = 10

sentiment_model.train()

for i in range(epochs):

  for data in train_dataloader:
    sentiment_model.zero_grad()
    senti_train = data["sentiment"].to(device)
    EMBED = BERT_MODEL(input_ids = data["input_ids"].to(device), attention_mask = data["attention_mask"].to(device)).pooler_output.to(device)
    _1, _2, senti_out = sentiment_model.forward(EMBED)

    loss_train = loss_func(EMBED, senti_out, senti_train)
    loss_train.backward()
    optimizer.step()

    sentiment_model.eval()

    with torch.no_grad():
      total_loss_val = 0
      for data in val_dataloader:
        senti_val = data["sentiment"].to(device)
        EMBED = BERT_MODEL(input_ids = data["input_ids"].to(device), attention_mask = data["attention_mask"].to(device)).pooler_output.to(device)
        _1, _2, senti_val_out = sentiment_model(EMBED)

        val_loss = loss_func(EMBED, senti_val_out, senti_val)

        total_val = senti_val.size(0)
        total_loss_val += val_loss.item()

      val_loss = total_loss_val / total_val

    if ((min_val_loss-val_loss) > 1e-4):
      min_val_loss = val_loss
      torch.save(sentiment_model.state_dict(), "bert_sent.pt")

  print(f"Epoch : {i+1}")
  print(f"Validation loss : {val_loss}")

  sentiment_model.load_state_dict(torch.load("bert_sent.pt"))

<ipython-input-12-aa0febf124e5>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "input_ids" : torch.tensor(self.input_ids[idx]).long().to(device),
<ipython-input-12-aa0febf124e5>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "attention_mask" : torch.tensor(self.attention_mask[idx]).long().to(device),


Epoch : 1
Validation loss : 5.244805172085762
Epoch : 2
Validation loss : 5.242507636547089
Epoch : 3
Validation loss : 5.240370005369186
Epoch : 4
Validation loss : 5.238311737775803
Epoch : 5
Validation loss : 5.236226201057434
Epoch : 6
Validation loss : 5.234210714697838
Epoch : 7
Validation loss : 5.232371240854263
Epoch : 8
Validation loss : 5.230563580989838
Epoch : 9
Validation loss : 5.22861710190773
Epoch : 10
Validation loss : 5.2267966121435165
Epoch : 11
Validation loss : 5.225081369280815
Epoch : 12
Validation loss : 5.223401352763176
Epoch : 13
Validation loss : 5.221796691417694
Epoch : 14
Validation loss : 5.220289051532745
Epoch : 15
Validation loss : 5.218810349702835
Epoch : 16
Validation loss : 5.217464968562126
Epoch : 17
Validation loss : 5.21613535284996
Epoch : 18
Validation loss : 5.214829623699188
Epoch : 19
Validation loss : 5.213618278503418
Epoch : 20
Validation loss : 5.212424859404564


In [ ]:
emotion_model = Emotion(classes = 7)
emotion_model = emotion_model.to(device)
emotion_model.train()

loss_func = SupConLoss()
optimizer = torch.optim.Adam(emotion_model.parameters(), lr = 1e-5, weight_decay = 0)
epochs = 20
min_val_loss = 10

emotion_model.train()

for i in range(epochs):

  for data in train_dataloader:
    emotion_model.zero_grad()
    emo_train = data["emotion"].to(device)
    EMBED = BERT_MODEL(input_ids = data["input_ids"].to(device), attention_mask = data["attention_mask"].to(device)).pooler_output.to(device)
    _1, _2, emo_out = emotion_model.forward(EMBED)

    loss_train = loss_func(EMBED, emo_out, emo_train)
    loss_train.backward()
    optimizer.step()

    emotion_model.eval()

    with torch.no_grad():
      total_loss_val = 0
      for data in val_dataloader:
        emo_val = data["emotion"].to(device)
        EMBED = BERT_MODEL(input_ids = data["input_ids"].to(device), attention_mask = data["attention_mask"].to(device)).pooler_output.to(device)
        _1, _2, emo_val_out = emotion_model(EMBED)

        val_loss = loss_func(EMBED, emo_val_out, emo_val)

        total_val = emo_val.size(0)
        total_loss_val += val_loss.item()

      val_loss = total_loss_val / total_val

    if ((min_val_loss-val_loss) > 1e-4):
      min_val_loss = val_loss
      torch.save(emotion_model.state_dict(), "bert_emo.pt")

  print(f"Epoch : {i+1}")
  print(f"Validation loss : {val_loss}")

  emotion_model.load_state_dict(torch.load("bert_emo.pt"))

<ipython-input-12-aa0febf124e5>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "input_ids" : torch.tensor(self.input_ids[idx]).long().to(device),
<ipython-input-12-aa0febf124e5>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "attention_mask" : torch.tensor(self.attention_mask[idx]).long().to(device),


Epoch : 1
Validation loss : 6.060157120227814
Epoch : 2
Validation loss : 5.990997314453125
Epoch : 3
Validation loss : 5.92701268196106
Epoch : 4
Validation loss : 5.905016511678696
Epoch : 5
Validation loss : 5.898145079612732
Epoch : 6
Validation loss : 5.895385384559631
Epoch : 7
Validation loss : 5.893817156553268
Epoch : 8
Validation loss : 5.892601311206818
Epoch : 9
Validation loss : 5.890933811664581
Epoch : 10
Validation loss : 5.887376829981804
Epoch : 11
Validation loss : 5.88085649907589
Epoch : 12
Validation loss : 5.877245903015137
Epoch : 13
Validation loss : 5.8749192506074905
Epoch : 14
Validation loss : 5.873162791132927
Epoch : 15
Validation loss : 5.871632695198059
Epoch : 16
Validation loss : 5.8703742027282715
Epoch : 17
Validation loss : 5.869419872760773
Epoch : 18
Validation loss : 5.868600413203239
Epoch : 19
Validation loss : 5.867877870798111
Epoch : 20
Validation loss : 5.867158189415932


In [ ]:
complaint_model = Complaint()
complaint_model = complaint_model.to(device)
complaint_model.train()

loss_func = SupConLoss()
optimizer = torch.optim.Adam(complaint_model.parameters(), lr = 1e-5, weight_decay = 0)
epochs = 20
min_val_loss = 10

complaint_model.train()

for i in range(epochs):

  for data in train_dataloader:
    complaint_model.zero_grad()
    comp_train = data["label"].to(device)
    sev_train = data["severity"].to(device)
    embed, comp_out, emo_out, senti_out, sev_out = complaint_model.forward(input_ids = data["input_ids"].to(device), attention_mask = data["attention_mask"].to(device))

    loss_train = loss_func(embed, comp_out, comp_train) + loss_func(embed, sev_out, sev_train)
    loss_train.backward()
    optimizer.step()

    complaint_model.eval()

    with torch.no_grad():
      total_loss_val = 0
      for data in val_dataloader:
        comp_val = data["label"].to(device)
        sev_val = data["severity"].to(device)
        embed, comp_val_out, emo_val_out, senti_val_out, sev_val_out = complaint_model(input_ids = data["input_ids"].to(device), attention_mask = data["attention_mask"].to(device))

        val_loss = loss_func(embed, comp_val_out, comp_val) + loss_func(embed, sev_val_out, sev_val)

        total_val = comp_val.size(0)
        total_loss_val += val_loss.item()

      val_loss = total_loss_val / total_val

    if ((min_val_loss-val_loss) > 1e-4):
      min_val_loss = val_loss
      torch.save(complaint_model.state_dict(), "bert_model.pt")

  print(f"Epoch : {i+1}")
  print(f"Validation loss : {val_loss}")

  complaint_model.load_state_dict(torch.load("bert_model.pt"))

In [ ]:
from sklearn.metrics import *

In [ ]:
!pip install torchmetrics

In [ ]:
from torchmetrics.classification import BinaryAccuracy, BinaryPrecision, BinaryF1Score, BinaryRecall
from torchmetrics.classification import MulticlassAccuracy, MulticlassPrecision, MulticlassF1Score, MulticlassRecall

comp_accuracy = BinaryAccuracy()
comp_precision = BinaryPrecision()
comp_f1 = BinaryF1Score()
comp_recall = BinaryRecall() 

sev_accuracy = MulticlassAccuracy(num_classes=5)
sev_precision = MulticlassPrecision(num_classes=5)
sev_f1 = MulticlassF1Score(num_classes=5)
sev_recall = MulticlassRecall(num_classes=5)

emo_accuracy = MulticlassAccuracy(num_classes=7)
emo_precision = MulticlassPrecision(num_classes=7)
emo_f1 = MulticlassF1Score(num_classes=7)
emo_recall = MulticlassRecall(num_classes=7)

senti_accuracy = MulticlassAccuracy(num_classes=3)
senti_precision = MulticlassPrecision(num_classes=3)
senti_f1 = MulticlassF1Score(num_classes=3)
senti_recall = MulticlassRecall(num_classes=3)

with torch.no_grad():
    for data in test_dataloader:
      lab_comp = data["label"]
      lab_emo = data["emotion"]
      lab_senti = data["sentiment"]
      lab_sev = data["severity"]

      comp_out, emo_out, senti_out, sev_out = complaint_model.forward(input_ids=data["input_ids"], attention_mask=data["attention_mask"])

      _, pred_comp = torch.max(comp_out.data,1)
      _, pred_emo = torch.max(emo_out.data,1)
      _, pred_senti = torch.max(senti_out.data,1)
      _, pred_sev = torch.max(sev_out.data,1)

      comp_accuracy.update(pred_comp.cpu(), lab_comp.cpu())
      comp_precision.update(pred_comp.cpu(), lab_comp.cpu()) 
      comp_f1.update(pred_comp.cpu(), lab_comp.cpu())
      comp_recall.update(pred_comp.cpu(), lab_comp.cpu())

      sev_accuracy.update(pred_sev.cpu(), lab_sev.cpu())
      sev_precision.update(pred_sev.cpu(), lab_sev.cpu())
      sev_f1.update(pred_sev.cpu(), lab_sev.cpu())
      sev_recall.update(pred_sev.cpu(), lab_sev.cpu())

      emo_accuracy.update(pred_emo.cpu(), lab_emo.cpu())
      emo_precision.update(pred_emo.cpu(), lab_emo.cpu())
      emo_f1.update(pred_emo.cpu(), lab_emo.cpu())
      emo_recall.update(pred_emo.cpu(), lab_emo.cpu())
      emo_test_accuracy = emo_accuracy.compute()

      senti_accuracy.update(pred_senti.cpu(), lab_senti.cpu())
      senti_precision.update(pred_senti.cpu(), lab_senti.cpu())
      senti_f1.update(pred_senti.cpu(), lab_senti.cpu())
      senti_recall.update(pred_senti.cpu(), lab_senti.cpu())
      senti_test_accuracy = senti_accuracy.compute()

    comp_test_accuracy = comp_accuracy.compute()
    comp_test_precision = comp_precision.compute()
    comp_test_f1 = comp_f1.compute()
    comp_test_recall = comp_recall.compute()

    sev_test_accuracy = sev_accuracy.compute()
    sev_test_precision = sev_precision.compute()
    sev_test_f1 = sev_f1.compute()
    sev_test_recall = sev_recall.compute()

    emo_test_accuracy = emo_accuracy.compute()
    emo_test_precision = emo_precision.compute()
    emo_test_f1 = emo_f1.compute()
    emo_test_recall = emo_recall.compute()

    senti_test_accuracy = senti_accuracy.compute()
    senti_test_precision = senti_precision.compute()
    senti_test_f1 = senti_f1.compute()
    senti_test_recall = senti_recall.compute()

    print(f"comp_test_accuracy : {comp_test_accuracy}")
    print(f"comp_test_precision : {comp_test_precision}")
    print(f"comp_test_f1 : {comp_test_f1}")
    print(f"comp_test_recall : {comp_test_recall}") 

    print(f"sev_test_accuracy : {sev_test_accuracy}")
    print(f"sev_test_precision : {sev_test_precision}")
    print(f"sev_test_f1 : {sev_test_f1}")
    print(f"sev_test_recall : {sev_test_recall}") 

    print(f"emo_test_accuracy : {emo_test_accuracy}")
    print(f"emo_test_precision : {emo_test_precision}")
    print(f"emo_test_f1 : {emo_test_f1}")
    print(f"emo_test_recall : {emo_test_recall}")

    print(f"senti_test_accuracy : {senti_test_accuracy}")
    print(f"senti_test_precision : {senti_test_precision}")
    print(f"senti_test_f1 : {senti_test_f1}")
    print(f"senti_test_recall : {senti_test_recall}")